# Demand and Travel Decision

## Generate a simple graph

### Flow Graph

In [ ]:
import matplotlib.pyplot as plt

from mnms.graph.generation import create_grid_graph
from mnms.tools.render import draw_flow_graph


mmgraph = create_grid_graph(2, 2)

fig, ax = plt.subplots(figsize=(16, 9))
draw_flow_graph(ax, mmgraph.flow_graph, linkwidth=2, nodesize=5)

### Mobility Graph

In [ ]:
from mnms.mobility_service.base import BaseMobilityService

car = BaseMobilityService('car', 10)

for node in mmgraph.flow_graph.nodes:
    car.add_node(f'C{node}', node)

for (up_node, down_node), link in mmgraph.flow_graph.links.items():
    car.add_link(f'C{up_node}_C{down_node}', 
                 f'C{up_node}', 
                 f'C{down_node}', 
                 costs={'length':1}, 
                 reference_links=[link.id])

mmgraph.add_mobility_service(car)

In [ ]:
from mnms.tools.render import draw_multimodal_graph

fig, ax = plt.subplots(figsize=(16, 9))
draw_multimodal_graph(ax, mmgraph, linkwidth=2, nodesize=5)

## Demand

### Random demand

In the following we create a random demand, the function `create_random_demand` return a `BaseDemandManager`.

In [ ]:
from mnms.demand.generation import create_random_demand

demand = create_random_demand(mmgraph, 
                              tstart="10:00:00", 
                              tend="12:00:00", 
                              cost_path='length', 
                              min_cost=4)
demand.show_users()

### CSV Predefined demand

In [ ]:
import pandas as pd

csv_file = pd.read_csv('data/simple_demand.csv', sep=';')
csv_file

In [ ]:
from mnms.demand import CSVDemandManager
from mnms.tools.time import Time

demand = CSVDemandManager('data/simple_demand.csv')

demand.get_next_departures(Time("07:00:00"), Time("07:30:00"))